# Classes Round Three

Each data point is a class instance.

- add get period, get only period to base class
- do cum sum stuff

In [1]:
# Import statements
import pandas as pd
import os
import datetime as dt
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoLocator
from itertools import product
import numpy as np
import csv

# Directories for reading and writing data
thedir = os.getcwd()
writedir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/interim'))
extdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/external'))
figdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'figures'))

# load data (reference data set of lcd and aws stations from 2018)
data_test = pd.read_csv(F'{extdir}/ref_2018.csv')
data_ = data_test[['Date_time_CET','BOLL_LCD_TEMP',
       'BOLL_AWS_TEMP', 'BOLL_PRECIP', 'BOLL_RADIATION_GLOBAL',
       'BOLL_WIND_SPEED_mean', 'ZOLL_STEVENSON_LCD_TEMP', 'ZOLL_2m_LCD_TEMP',
       'ZOLL_3m_LCD_TEMP', 'ZOLL_AWS_TEMP', 'ZOLL_RADIATION_GLOBAL',
       'ZOLL_SUNSHINE', 'ZOLL_PRECIP', 'ZOLL_WIND_SPEED_MEAN', 'AFU_LCD_TEMP',
       'AFU_AWS_TEMP', 'AFU_WIND_SPEED_MEAN', 'AFU_RADIATION_GLOBAL']]
data_.set_index('Date_time_CET',inplace=True)

In [2]:
class Thing:
    def __init__(self, value, index, header):
        self.name = header
        self.time = dt.datetime.strptime(index,'%d.%m.%Y %H:%M')
        self.value = value
        self.station = header[:3].lower()
        if 'LCD' in header:
            self.stype = 'lcd'
        else:
            self.stype = 'aws'
        if 'TEMP' in header:
            self.qty = 'temp'
        elif 'SUN':
            self.qty = 'sun'
        elif 'RADI':
            self.qty = 'rad'
        elif 'WIND':
            self.qty = 'wind'
        elif 'PREC':
            self.qty = 'prec'
    
    def __lt__(self, other):
         return self.time < other.time
    
    #def time_date(self):
    #    return dt.datetime.strptime(self.time,'%d.%m.%Y %H:%M')
    
    def get_period(self, strfkey):
        return dt.datetime.strftime(self.time,strfkey)
    
    def update_value(self,value):
        self.value = value
    
    def return_name(self):
        return self.name
    
    def return_value(self):
        return self.value
    
    def return_station(self):
        return self.station
    
    def return_time(self):
        return self.time
    
    def return_qty(self):
        return self.qty
    
    

In [3]:
cols = data_.columns
mylist = []
for index,rows in data_.iterrows():
        mylist.append([Thing(data_.at[index,index_c],index,index_c) for index_c in cols])

In [4]:
ndf = np.array([ x for y in mylist for x in y])
get_quant = np.vectorize(lambda x: x.qty,otypes = [object])

In [6]:
len(mylist)

17712

In [ ]:

period_keys = {'hour':'%H','month':'%m','year':'%Y','day':'%j'}
qty = ['temp','wind','rad','sun','prec']
station = ['bol','zol','afu']
key = period_keys['day']

temp lcd - aws differences

In [ ]:
def get_diffs(station_dicts):
    newdict = {}
    for key,values in station_dict.items():
        val = values[0]-values[1]
        newdict[key] = [val,values[2]]
    return newdict

def get_dicts(stations,qty):
    tdict = {}
    for i in stations:
        newdict = {}
        lcds = set([x.name for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'lcd')])
        num_lcd = len(lcds)
        aws = [(x.value,x.time,x.name) for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'aws')] 
        lcd = [(x.value,x.time,x.name) for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'lcd')]
        aws_= aws*num_lcd
        total = list(zip(aws_,lcd))
        aws = [(x.value,x.time,x.name) for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'aws')] 
        lcd = [(x.value,x.time,x.name) for x in ndf if (x.qty == qty) & (x.station == i) & (x.stype == 'lcd')]
        total = list(zip(aws,lcd))
        for j in lcds:
            tdict = {}
            for k in total:
                newdict[k[0][1]] = [k[0][0],k[1][0],j]
            holder = [x[0] for x in newdict if j in x.values()]
            tdict[j] = holder
    return tdict

def make_frames(stations,qty):
    dicts = get_dicts(stations,qty)
    vdicts = get_diffs(dicts)
    df = pd.DataFrame.from_dict(vdicts, orient = 'index')
    return df

In [ ]:
test = get_dicts(['bol','zol','afu'],'temp')

In [ ]:
len(test['ZOLL_AWS_TEMP-ZOLL_3m_LCD_TEMP'])

In [ ]:
test.keys()